In [10]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as cm
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import sklearn
import shapely.wkt
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import svm
import hdbscan # new clustering algorithm
import mplleaflet
from scipy.spatial.distance import pdist, squareform, euclidean, directed_hausdorff
from haversine import haversine


%matplotlib inline

In [2]:
gdf = gpd.read_file("../app_route_data/trajets_mtl_trajet_2017.shp", encoding='utf-8') # utf-8 needed to read french letters
mtl_region = gpd.read_file("../shapes/mtl_all_regions.json")
mtl_dissem = gpd.read_file("../shapes/mtl_dissem.geojson")
mtl_greater = gpd.read_file("../shapes/greater_montreal.geojson")
city_of_montreal = mtl_region.loc[(mtl_region.AIRE > 0) & (mtl_region.TYPE == 'Arrondissement')]

In [3]:
mtl_greater['geometry'] = mtl_greater.buffer(0.01)
mtl_greater = mtl_greater.dissolve(by="CMANAME")

In [4]:
# outlier removal
gdf = gdf.loc[gdf.distance_m >= 50]
gdf = gdf.loc[gdf.seconds >= 60]
gdf = gdf.loc[gdf.distance_m <= 100000]
gdf = gdf.loc[gdf.seconds <= 10800]
gdf = gdf.dropna(subset=['mode'])
gdf = gdf.reset_index(drop=True)

In [5]:
# turn the data back into datetime
gdf['starttime'] = gdf['starttime'].apply(gpd.pd.to_datetime)
gdf['endtime'] = gdf['endtime'].apply(gpd.pd.to_datetime)

In [6]:
gdf['log_dur'] = np.log(gdf.seconds+1)
gdf['log_dist'] = np.log(gdf.distance_m+1)

In [7]:
new_classifications = {"1":["shops","leisure","cafe","returning_home"],"2":["education"],\
                       "3":["health"],"4":["pick_up_a_person"],"5":["work"]}

In [8]:
def new_pur_labels(row, purpose_label_dict=new_classifications):
    for key, val in purpose_label_dict.items():
        if row.purpose in val:
            return key

    return None

In [9]:
gdf['purpose_labels'] = gdf.apply(new_pur_labels, axis=1)
gdf.purpose_labels.value_counts()

1    47464
5    18950
2     2769
4     1574
3     1044
Name: purpose_labels, dtype: int64

In [12]:
gdf = gdf.dropna(subset=["purpose_labels"])

## Evaluate

In [154]:
joined_mode_data = gpd.read_file("analysis_results/joined_model_data.shp")

In [155]:
gdf.purpose.value_counts()

returning_home      26819
work                18950
leisure              9167
shops                8363
cafe                 3115
education            2769
pick_up_a_person     1574
health               1044
Name: purpose, dtype: int64

In [156]:
joined_mode_data.purpose_la = joined_mode_data.purpose_la.astype(float)

In [157]:
def check_scores(row):
    svc_correct = int(row['purpose_la']) == int(row['svcall'])
    rf_correct = int(row['rf' + str(int(row['purpose_la']))]) == 1
    nn_correct = int(row['nn' + str(int(row['purpose_la']))]) == 1
    return svc_correct, rf_correct, nn_correct

In [158]:
check_scores(joined_mode_data.loc[0])

(False, True, True)

In [160]:
joined_mode_data["which_correct"] = joined_mode_data.apply(check_scores, axis=1)

In [161]:
each_correct = joined_mode_data["which_correct"].apply(pd.Series)
each_correct.columns = ["svc_corr","rf_corr","nn_corr"]
joined_mode_data = pd.concat([joined_mode_data, each_correct], axis=1)

In [162]:
all_true = joined_mode_data.loc[(joined_mode_data.svc_corr == True)&(joined_mode_data.rf_corr == True)&(joined_mode_data.nn_corr == True)]
no_true = joined_mode_data.loc[(joined_mode_data.svc_corr != True)&(joined_mode_data.rf_corr != True)&(joined_mode_data.nn_corr != True)]
only_svc = joined_mode_data.loc[(joined_mode_data.svc_corr == True)&(joined_mode_data.rf_corr != True)&(joined_mode_data.nn_corr != True)]
only_rf = joined_mode_data.loc[(joined_mode_data.svc_corr != True)&(joined_mode_data.rf_corr == True)&(joined_mode_data.nn_corr != True)]
only_nn = joined_mode_data.loc[(joined_mode_data.svc_corr != True)&(joined_mode_data.rf_corr != True)&(joined_mode_data.nn_corr == True)]
all_but_svc = joined_mode_data.loc[(joined_mode_data.svc_corr != True)&(joined_mode_data.rf_corr == True)&(joined_mode_data.nn_corr == True)]
all_but_rf = joined_mode_data.loc[(joined_mode_data.svc_corr == True)&(joined_mode_data.rf_corr != True)&(joined_mode_data.nn_corr == True)]
all_but_nn = joined_mode_data.loc[(joined_mode_data.svc_corr == True)&(joined_mode_data.rf_corr == True)&(joined_mode_data.nn_corr != True)]



In [169]:
len(all_true), len(no_true), len(only_svc), len(only_rf), len(only_nn), len(all_but_svc),len(all_but_rf),len(all_but_nn)



(13773, 3801, 857, 218, 588, 3427, 848, 183)

In [164]:
all_true.purpose.value_counts()

returning_home    8042
leisure           2663
shops             2281
cafe               787
Name: purpose, dtype: int64

In [165]:
no_true.purpose.value_counts()

work                2019
education            951
pick_up_a_person     509
health               322
Name: purpose, dtype: int64

In [168]:
only_nn.purpose.value_counts()

work    588
Name: purpose, dtype: int64

In [166]:
only_svc.purpose.value_counts(), only_rf.purpose.value_counts(), only_nn.purpose.value_counts()

(returning_home    347
 shops             227
 leisure           174
 cafe              109
 Name: purpose, dtype: int64, work                213
 education             4
 pick_up_a_person      1
 Name: purpose, dtype: int64, work    588
 Name: purpose, dtype: int64)

In [170]:
joined_mode_data[["rf1","rf2","rf3","rf4","rf5"]].sum(axis=1).value_counts()

1.0    21108
0.0     2419
2.0      167
3.0        1
dtype: int64

In [171]:
joined_mode_data[["nn1","nn2","nn3","nn4","nn5"]].sum(axis=1).value_counts()

1.0    23136
0.0      559
dtype: int64

In [172]:
max_ind = []
for row in joined_mode_data[["nn1","nn2","nn3","nn4","nn5"]].iterrows():
    max_ind.append(np.where(row[1].values == np.amax(row[1].values))[0]+1)
joined_mode_data['nn_pred'] = [float(val[0]) if len(val) == 1 else 0.0 for val in max_ind]

In [173]:
max_ind = []
for row in joined_mode_data[["rf1","rf2","rf3","rf4","rf5"]].iterrows():
    max_ind.append(np.where(row[1].values == np.amax(row[1].values))[0]+1)
joined_mode_data['rf_pred'] = [float(val[0]) if len(val) == 1 else 0.0 for val in max_ind]

In [174]:
from sklearn.metrics import classification_report
print('Results on the test set: RF:')
print(classification_report(joined_mode_data.purpose_la, joined_mode_data.rf_pred))
print('Results on the test set: SVC:')
print(classification_report(joined_mode_data.purpose_la, joined_mode_data.svcall))
print('Results on the test set: NN:')
print(classification_report(joined_mode_data.purpose_la, joined_mode_data.nn_pred))

Results on the test set: RF:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       0.85      0.89      0.87     15661
         2.0       0.00      0.00      0.00       955
         3.0       0.00      0.00      0.00       322
         4.0       1.00      0.00      0.00       510
         5.0       0.76      0.57      0.65      6247

    accuracy                           0.74     23695
   macro avg       0.43      0.24      0.25     23695
weighted avg       0.78      0.74      0.74     23695

Results on the test set: SVC:
              precision    recall  f1-score   support

         1.0       0.66      1.00      0.80     15661
         2.0       0.00      0.00      0.00       955
         3.0       0.00      0.00      0.00       322
         4.0       0.00      0.00      0.00       510
         5.0       0.00      0.00      0.00      6247

    accuracy                           0.66     23695
   macro avg      

/Applications/anaconda/envs/st-ds/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Applications/anaconda/envs/st-ds/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(False, False, False)

In [112]:
joined_mode_data

,id_trip,mode,purpose,starttime,endtime,seconds,distance_m,direction,magnitude,carddir,...,rf5,svcall,nn1,nn2,nn3,nn4,nn5,geometry,nn_pred,rf_pred
0,358412,3,work,2017-09-18 07:27:38-04:00,2017-09-18 07:40:26-04:00,768,992.314303,211.576183,0.347807,2,...,1.0,5.0,0.0,0.0,0.0,0.0,1.0,LINESTRING (7629026.972989381 1247452.25399864...,5.0,0.0
1,35763,0,shops,2017-09-18 07:32:54-04:00,2017-09-18 08:04:07-04:00,1873,6832.113937,338.681845,0.002750,3,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,LINESTRING (7641918.591122856 1236660.83022967...,5.0,1.0
2,230363,2,work,2017-09-18 07:33:37-04:00,2017-09-18 08:16:09-04:00,2552,8500.854359,127.802412,0.098128,4,...,1.0,5.0,0.0,0.0,0.0,0.0,1.0,(LINESTRING (7625449.252399736 1247376.6710083...,5.0,5.0
3,143596,7,work,2017-09-18 10:39:19-04:00,2017-09-18 10:55:09-04:00,950,2642.961059,349.675299,0.185333,13,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,(LINESTRING (7630815.520267755 1246190.8794987...,5.0,5.0
4,313223,3,work,2017-09-18 11:29:35-04:00,2017-09-18 11:31:54-04:00,139,226.672022,66.405847,0.252944,5,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,LINESTRING (7631532.876323251 1245163.99977121...,0.0,0.0
5,353958,0,work,2017-09-18 11:45:48-04:00,2017-09-18 12:10:22-04:00,1474,8685.133631,308.020841,0.073248,7,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,LINESTRING (7628061.786595868 1240569.55374326...,5.0,1.0
6,44664,4,work,2017-09-18 11:57:12-04:00,2017-09-18 12:17:29-04:00,1217,3228.128354,155.607434,0.189776,8,...,1.0,5.0,0.0,0.0,0.0,0.0,1.0,LINESTRING (7626422.447991624 1246687.45536286...,5.0,5.0
7,474247,0,work,2017-09-18 12:57:40-04:00,2017-09-18 13:21:00-04:00,1400,445.897686,91.763257,0.320339,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,LINESTRING (7631172.703754629 1254786.97950467...,1.0,1.0
8,443357,3,shops,2017-09-18 13:04:11-04:00,2017-09-18 13:22:58-04:00,1127,650.601955,19.992785,0.164362,6,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,LINESTRING (7631364.037662272 1244778.99428346...,1.0,0.0
9,135984,7,returning_home,2017-09-18 13:15:17-04:00,2017-09-18 13:29:09-04:00,832,1979.418259,161.115251,0.214882,8,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,(LINESTRING (7629893.274594538 1247934.1841953...,1.0,1.0


In [111]:
joined_mode_data.apply(check_scores, axis=1)

KeyError: ('purpose_la', 'occurred at index id_trip')

In [25]:
joined_mode_data

,id_trip,mode,purpose,starttime,endtime,seconds,distance_m,direction,magnitude,carddir,...,rf3,rf4,rf5,svcall,nn1,nn2,nn3,nn4,nn5,geometry
0,358412,3,work,2017-09-18 07:27:38-04:00,2017-09-18 07:40:26-04:00,768,992.314303,211.576183,0.347807,2,...,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0,LINESTRING (7629026.972989381 1247452.25399864...
1,35763,0,shops,2017-09-18 07:32:54-04:00,2017-09-18 08:04:07-04:00,1873,6832.113937,338.681845,0.002750,3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,LINESTRING (7641918.591122856 1236660.83022967...
2,230363,2,work,2017-09-18 07:33:37-04:00,2017-09-18 08:16:09-04:00,2552,8500.854359,127.802412,0.098128,4,...,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0,(LINESTRING (7625449.252399736 1247376.6710083...
3,143596,7,work,2017-09-18 10:39:19-04:00,2017-09-18 10:55:09-04:00,950,2642.961059,349.675299,0.185333,13,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,(LINESTRING (7630815.520267755 1246190.8794987...
4,313223,3,work,2017-09-18 11:29:35-04:00,2017-09-18 11:31:54-04:00,139,226.672022,66.405847,0.252944,5,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,LINESTRING (7631532.876323251 1245163.99977121...
5,353958,0,work,2017-09-18 11:45:48-04:00,2017-09-18 12:10:22-04:00,1474,8685.133631,308.020841,0.073248,7,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,LINESTRING (7628061.786595868 1240569.55374326...
6,44664,4,work,2017-09-18 11:57:12-04:00,2017-09-18 12:17:29-04:00,1217,3228.128354,155.607434,0.189776,8,...,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0,LINESTRING (7626422.447991624 1246687.45536286...
7,474247,0,work,2017-09-18 12:57:40-04:00,2017-09-18 13:21:00-04:00,1400,445.897686,91.763257,0.320339,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,LINESTRING (7631172.703754629 1254786.97950467...
8,443357,3,shops,2017-09-18 13:04:11-04:00,2017-09-18 13:22:58-04:00,1127,650.601955,19.992785,0.164362,6,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,LINESTRING (7631364.037662272 1244778.99428346...
9,135984,7,returning_home,2017-09-18 13:15:17-04:00,2017-09-18 13:29:09-04:00,832,1979.418259,161.115251,0.214882,8,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,(LINESTRING (7629893.274594538 1247934.1841953...


In [ ]:
precision_recall_fscore_support(y_true, y_pred, average='weighted')